In [1]:
import os
import tensorflow as tf
import numpy as np
import tempfile

image_size = 64 # 64 x 64 image
total_epochs = 50
batch_size = 100

In [2]:

def deepcnn_dogcat(x):
    """deepcnn_dogcat builds the computational graph for a NN that distinguishes cats and dogs.
    Args:
    x: an input tensor with the dimensions (N_examples, image_size, image_size, 3)
    Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 2) containing the logit
    values for the two classes (before softmax activation). 
    """
    x_image = tf.reshape(x, [-1, image_size, image_size, 3])

    conv1_layer = conv2d_maxpool(x_image, 3, 32, 2, name="conv1")
    
    conv2_layer = conv2d_maxpool(conv1_layer, 32, 64, 2, name="conv2")
    
    conv3_layer = conv2d_maxpool(conv2_layer, 64, 64, 2, name="conv3")
    
    conv4_layer = conv2d_maxpool(conv3_layer, 64, 64, 2, name="conv4")
    
    flat5_layer = tf.reshape(conv4_layer, [-1, 4*4*64])
    full5_layer = tf.nn.relu(fullcon(flat5_layer, 4*4*64, 256, name="fullcon5"))
    
    drop_layer = tf.nn.dropout(full5_layer, 0.6)
    out_layer = fullcon(drop_layer, 256, 2, name="output")
    
    return out_layer

def conv2d_maxpool(x, channels_in, channels_out, strides, name="conv2d"):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal(shape=[3, 3, channels_in, channels_out], stddev=0.1, name="W"))
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, W)
        b = tf.Variable(tf.constant(0.1, shape=[channels_out], name="b"))
        conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv + b)
        pool = tf.nn.max_pool(act, ksize=[1, strides, strides, 1], strides=[1, strides, strides, 1], padding='SAME')
        #tf.summary.histogram("weights", W)
        #tf.summary.histogram("biases", b)
        #tf.summary.histogram("activations", act)
        return pool

def fullcon(x, channels_in, channels_out, name="fullcon"):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal(shape=[channels_in, channels_out], stddev=0.1, name="W"))
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, W)
        b = tf.Variable(tf.constant(0.1, shape=[channels_out], name="b"))
        y = tf.matmul(x, W) + b
        #tf.summary.histogram("weights", W)
        #tf.summary.histogram("biases", b)
        #tf.summary.histogram("pre-activations", y)
        return y


In [3]:
# Placeholder for the input
x = tf.placeholder(tf.float32, [None, image_size, image_size, 3], name="x")
#tf.summary.image('input', x, 3)

# Define loss and optimizers
y_ = tf.placeholder(tf.float32, [None, 2], name="labels")

regularizer = tf.contrib.layers.l2_regularizer(scale=0.001)

# Build the graph for the deep net
y_conv = deepcnn_dogcat(x)

with tf.name_scope('loss'):
    reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_loss = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)
    cross_entropy += reg_loss
    tf.summary.scalar("reg_cost", reg_loss)
    tf.summary.scalar("total_cost", cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)
    tf.summary.scalar("accuracy", accuracy)

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

train_writer = tf.summary.FileWriter("./logs/train/")
test_writer = tf.summary.FileWriter("./logs/test/")
train_writer.add_graph(sess.graph)
summ = tf.summary.merge_all()

In [7]:
#TRAIN A NEURAL NETWORK. DONT RUN IF YOU WANT TO JUST TEST.

X = np.load('train_data_' + str(image_size) + '.npy')
Y = np.load('train_labels_' + str(image_size) + '.npy')
X_valid = np.load('valid_data_' + str(image_size) + '.npy')
Y_valid = np.load('valid_labels_' + str(image_size) + '.npy')

saver = tf.train.Saver()
iterations = 0
validacc_latest = 0
for i in range(total_epochs):
    for j in range(0,X.shape[0]-batch_size,batch_size):
        sess.run(train_step, feed_dict={x: X[j:j+batch_size], y_: Y[j:j+batch_size]})
        iterations += 1
        if (iterations % 100 == 0):
            summary, train_accuracy = sess.run([summ,accuracy], feed_dict={x: X[j:j+batch_size], y_: Y[j:j+batch_size]})
            train_writer.add_summary(summary, iterations)
            summary, valid_accuracy = sess.run([summ,accuracy], feed_dict={x: X_valid, y_: Y_valid})
            test_writer.add_summary(summary, iterations)
            if (valid_accuracy > validacc_latest):
                save_path = saver.save(sess, "./cnn_model.ckpt")
                print("Model saved in file: %s" % save_path)
                validacc_latest = valid_accuracy
            print('epoch %d, iterations %d, training accuracy %g, validation accuracy %g' % (i, iterations, train_accuracy, valid_accuracy))

saver = tf.train.Saver()
save_path = saver.save(sess, "./cnn_model.ckpt")
print("Model saved in file: %s" % save_path)

sess.close()

Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 100, training accuracy 0.5, validation accuracy 0.59808
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 200, training accuracy 0.55, validation accuracy 0.64096
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 300, training accuracy 0.65, validation accuracy 0.67296
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 400, training accuracy 0.69, validation accuracy 0.67936
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 500, training accuracy 0.72, validation accuracy 0.69888
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 600, training accuracy 0.64, validation accuracy 0.70976
epoch 0, iterations 700, training accuracy 0.69, validation accuracy 0.70208
Model saved in file: ./cnn_model.ckpt
epoch 0, iterations 800, training accuracy 0.71, validation accuracy 0.72256
epoch 0, iterations 900, training accuracy 0.65, validation accuracy 0.7152
Model saved in file: ./cnn_model.ckpt
epoch

epoch 7, iterations 9200, training accuracy 0.8, validation accuracy 0.81312
Model saved in file: ./cnn_model.ckpt
epoch 7, iterations 9300, training accuracy 0.88, validation accuracy 0.82368
Model saved in file: ./cnn_model.ckpt
epoch 7, iterations 9400, training accuracy 0.89, validation accuracy 0.82912
epoch 8, iterations 9500, training accuracy 0.79, validation accuracy 0.7952
epoch 8, iterations 9600, training accuracy 0.82, validation accuracy 0.82304
epoch 8, iterations 9700, training accuracy 0.9, validation accuracy 0.81856
epoch 8, iterations 9800, training accuracy 0.77, validation accuracy 0.8176
epoch 8, iterations 9900, training accuracy 0.8, validation accuracy 0.82208
epoch 8, iterations 10000, training accuracy 0.87, validation accuracy 0.82528
epoch 8, iterations 10100, training accuracy 0.89, validation accuracy 0.82304
epoch 8, iterations 10200, training accuracy 0.81, validation accuracy 0.82496
epoch 8, iterations 10300, training accuracy 0.81, validation accura

epoch 15, iterations 18900, training accuracy 0.85, validation accuracy 0.84832
epoch 16, iterations 19000, training accuracy 0.86, validation accuracy 0.85248
Model saved in file: ./cnn_model.ckpt
epoch 16, iterations 19100, training accuracy 0.83, validation accuracy 0.85824
epoch 16, iterations 19200, training accuracy 0.83, validation accuracy 0.8496
epoch 16, iterations 19300, training accuracy 0.85, validation accuracy 0.84768
epoch 16, iterations 19400, training accuracy 0.92, validation accuracy 0.85536
epoch 16, iterations 19500, training accuracy 0.82, validation accuracy 0.85664
epoch 16, iterations 19600, training accuracy 0.85, validation accuracy 0.85216
epoch 16, iterations 19700, training accuracy 0.88, validation accuracy 0.84608
epoch 16, iterations 19800, training accuracy 0.89, validation accuracy 0.85056
epoch 16, iterations 19900, training accuracy 0.88, validation accuracy 0.83904
epoch 16, iterations 20000, training accuracy 0.88, validation accuracy 0.84992
epo

epoch 24, iterations 29000, training accuracy 0.95, validation accuracy 0.85408
epoch 24, iterations 29100, training accuracy 0.91, validation accuracy 0.8528
epoch 24, iterations 29200, training accuracy 0.89, validation accuracy 0.85824
epoch 24, iterations 29300, training accuracy 0.91, validation accuracy 0.86368
epoch 24, iterations 29400, training accuracy 0.93, validation accuracy 0.8592
epoch 24, iterations 29500, training accuracy 0.85, validation accuracy 0.86656
epoch 24, iterations 29600, training accuracy 0.87, validation accuracy 0.85888
epoch 25, iterations 29700, training accuracy 0.93, validation accuracy 0.8608
epoch 25, iterations 29800, training accuracy 0.89, validation accuracy 0.86656
epoch 25, iterations 29900, training accuracy 0.92, validation accuracy 0.86112
epoch 25, iterations 30000, training accuracy 0.93, validation accuracy 0.8656
epoch 25, iterations 30100, training accuracy 0.9, validation accuracy 0.86112
epoch 25, iterations 30200, training accuracy

epoch 32, iterations 39100, training accuracy 0.9, validation accuracy 0.85888
epoch 33, iterations 39200, training accuracy 0.89, validation accuracy 0.86272
Model saved in file: ./cnn_model.ckpt
epoch 33, iterations 39300, training accuracy 0.93, validation accuracy 0.87488
epoch 33, iterations 39400, training accuracy 0.9, validation accuracy 0.86784
epoch 33, iterations 39500, training accuracy 0.94, validation accuracy 0.872
epoch 33, iterations 39600, training accuracy 0.91, validation accuracy 0.87104
epoch 33, iterations 39700, training accuracy 0.91, validation accuracy 0.86336
Model saved in file: ./cnn_model.ckpt
epoch 33, iterations 39800, training accuracy 0.92, validation accuracy 0.87648
epoch 33, iterations 39900, training accuracy 0.9, validation accuracy 0.86944
epoch 33, iterations 40000, training accuracy 0.93, validation accuracy 0.86944
epoch 33, iterations 40100, training accuracy 0.92, validation accuracy 0.8736
epoch 33, iterations 40200, training accuracy 0.86

Model saved in file: ./cnn_model.ckpt
epoch 41, iterations 49300, training accuracy 0.89, validation accuracy 0.87872
epoch 41, iterations 49400, training accuracy 0.95, validation accuracy 0.86752
epoch 41, iterations 49500, training accuracy 0.96, validation accuracy 0.86624
epoch 41, iterations 49600, training accuracy 0.94, validation accuracy 0.87584
epoch 41, iterations 49700, training accuracy 0.91, validation accuracy 0.8528
epoch 41, iterations 49800, training accuracy 0.97, validation accuracy 0.87456
epoch 42, iterations 49900, training accuracy 0.95, validation accuracy 0.87776
epoch 42, iterations 50000, training accuracy 0.98, validation accuracy 0.86976
epoch 42, iterations 50100, training accuracy 0.94, validation accuracy 0.87104
epoch 42, iterations 50200, training accuracy 0.93, validation accuracy 0.87872
epoch 42, iterations 50300, training accuracy 0.95, validation accuracy 0.84704
epoch 42, iterations 50400, training accuracy 0.94, validation accuracy 0.8784
epoc

In [4]:
#ACCURACY TEST ON TEST DATA

X_test = np.load('test_data_' + str(image_size) + '.npy')
Y_test = np.load('test_labels_' + str(image_size) + '.npy')

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, "./cnn_model.ckpt")
perf = sess.run(accuracy, feed_dict={x: X_test, y_: Y_test})
print('test set accuracy %g' % perf)
#print(type(perf))
sess.close()

INFO:tensorflow:Restoring parameters from ./cnn_model.ckpt
test set accuracy 0.84032
